# Chapter 4: Building Good Training Data Sets - Data Preprocessing

# 4-1: Dealing with Missing Data

Most computational models cannot handle missing values in a data set, and would produce unpredicatable results if not addressed.

In [4]:
import pandas as pd
from io import StringIO
csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''
csv_data=unicode(csv_data)
df=pd.read_csv(StringIO(csv_data))
print df

df.isnull().sum()

      A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   NaN  8.0
2  10.0  11.0  12.0  NaN


A    0
B    0
C    1
D    1
dtype: int64

### Eliminating samples or features with missing values

In [5]:
# drop observations with Null values
null_obs=df.dropna()

# drop columns with any null values
null_col=df.dropna(axis=1)

# only drop rows were all columns are NaN
null_col_all=df.dropna(how='all')

# drop rows that do not have at least 4 non-NaN values
at_least = df.dropna(thresh=4)

# only drop rows where NaN appears in specific columns (here: 'C')
drop_specific = df.dropna(subset=['C'])

### Imputing missing values

When dropping NaNs is not feasible, youc an use different interpolation techniques to estimate the missing values from the other training samples in your dataset.

In [9]:
from sklearn.preprocessing import Imputer
imr = Imputer(missing_values='NaN', strategy = 'mean', axis=0)
imr = imr.fit(df)
imputed_data = imr.transform(df.values)
imputed_data

array([[  1. ,   2. ,   3. ,   4. ],
       [  5. ,   6. ,   7.5,   8. ],
       [ 10. ,  11. ,  12. ,   6. ]])

### Understanding scikit-learn *transformer* classes

The two essential methods of estimators in this class are:

**(1) fit**: Used to learn the parametrs from the training data

**(2) transform**: Uses those parameters to transform the data

*Note: the classifiers from Chapter 3 belong to the **estimator** class within scikit-learn, and have a **predict** method*

# 4-2: Handling categorical data

First, understand **nominal** vs. **ordinal** features. Ordinal features can be sorted (i.e. t-shirt sizes), while nominal features do not imply an order (i.e. t-shirt colors).

We'll create a new df to illustrate:

In [33]:
import pandas as pd
df = pd.DataFrame([
    ['green','M',10.1,'class1'],
    ['red','L',13.5,'class2'],
    ['blue','XL',15.3,'class1']])
df.columns = ['color','size','price','classlabel']
df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


### Mapping ordinal features

Mapping an oridinal feature to an integer is a manual process, and we must scale the difference in values accordingly

In [34]:
size_mapping = {
    'XL':3,
    'L':2,
    'M':1}
df['size'] = df['size'].map(size_mapping)
print df

# You can always define and inverse mapping to revert to original labels

inv_size_mapping = {v: k for k, v in size_mapping.items()}

   color  size  price classlabel
0  green     1   10.1     class1
1    red     2   13.5     class2
2   blue     3   15.3     class1


### Encoding Class Labels

While class labels are *not* ordinal, many algorithms still require these labels are passed as integers. You can begin coding at 0, though this will be suboptimal with greater than 2 features

In [28]:
import numpy as np
class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
print class_mapping

df['classlabel'] = df['classlabel'].map(class_mapping)

df

{'class2': 1, 'class1': 0}


,color,size,price,classlabel
0,green,1,10.1,0
1,red,2,13.5,1
2,blue,3,15.3,0


#### Inverse Mapping

In [8]:
#inv_class_mapping = {v: k for k,v in class_mapping.items()}
#df['classlabel'] = df['classlabel'].map(inv_class_mapping)
#df

,color,size,price,classlabel
0,green,M,10.1,class1
1,red,L,13.5,class2
2,blue,XL,15.3,class1


#### Or, use the LabelEncoder

In [29]:
from sklearn.preprocessing import LabelEncoder
class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
#fit_transform is a shorcut for calling fit and transform separately

y

array([0, 1, 0])

#### And inverse...

In [11]:
# class_le.inverse_transform(y)

array(['class1', 'class2', 'class1'], dtype=object)

### Performing one-hot encoding on nominal features

In the case where you have multiple classifiers, you'll want to create a new **dummy feature** for reach unique value rather than convert to sequential integers as before. This is called **one-hot** encoding.

The old way:

In [35]:
X = df[['color','size','price']].values
color_le = LabelEncoder()
X[:,0] = color_le.fit_transform(X[:,0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

The new way:

In [36]:

from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(categorical_features=[0])
ohe.fit_transform(X).toarray()

array([[  0. ,   1. ,   0. ,   1. ,  10.1],
       [  0. ,   0. ,   1. ,   2. ,  13.5],
       [  1. ,   0. ,   0. ,   3. ,  15.3]])

When initializing the `OneHotEncoder`, we define the column position of the variable that we want to transform via `categorical_features`. By default, the resulting sparse matrix must be converted to a *dense* **NumpyArray** via `toarray()`. To skip the `toarray()`, you could initialize the encoder as `OneHotEncoder(...,sparse=False)` instead.

### Or even better... just use `GetDummies`!

In [38]:
pd.get_dummies(df[['price','color','size']])

,price,size,color_blue,color_green,color_red
0,10.1,1,0,1,0
1,13.5,2,0,0,1
2,15.3,3,1,0,0


# 4-3: Partitioning a dataset in training and test sets